In [ ]:
import pandas as pd
import requests
import json
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np

In [ ]:
# Load training and validation datasets
zone_1_training_data = pd.read_csv("seismic_zone_1.csv")
zone_1_val_data = pd.read_csv("seismic_zone_1_val.csv")

zone_2_training_data = pd.read_csv("seismic_zone_2.csv")
zone_2_val_data = pd.read_csv("seismic_zone_2_val.csv")

In [ ]:
# Define function for loading validation data
FEATURES = ["depth", "mag", "gap", "dmin", "rms"]
def load_val_data(df):
    X_test = df[FEATURES] 
    y_true = df[["latitude", "longitude"]]
    
    return X_test, y_true

# API calls
def predict(url, X_test):
    API_URL = f'{url}/predict'

    predictions = []

    for _, row in X_test.iterrows():
        payload = row.to_dict()
        
        response = requests.post(API_URL, json=payload)
        
        if response.status_code == 200:
            pred = response.json()["prediction"]
            predictions.append(pred[0])
        else:
            print(f"Error for input {row}: {response.text}")
            
    return predictions

def evaluate_predictions(model_response, true_value):
    y_pred = pd.DataFrame(model_response, columns=["latitude", "longitude"])
    
    y_pred = pd.DataFrame(model_response, columns=["latitude", "longitude"])
    true_value = true_value.reset_index(drop=True)

    mse_lat = mean_squared_error(true_value["latitude"], y_pred["latitude"])
    mse_lon = mean_squared_error(true_value["longitude"], y_pred["longitude"])

    r2_lat = r2_score(true_value["latitude"], y_pred["latitude"])
    r2_lon = r2_score(true_value["longitude"], y_pred["longitude"])

    mae_lat = mean_absolute_error(true_value["latitude"], y_pred["latitude"])
    mae_lon = mean_absolute_error(true_value["longitude"], y_pred["longitude"])

    print(f"Mean Squared Error (Latitude): {mse_lat}")
    print(f"Mean Squared Error (Longitude): {mse_lon}\n")

    print(f"R² Score (Latitude): {r2_lat}")
    print(f"R² Score (Longitude): {r2_lon}\n")

    print(f"Mean Absolute Error (Latitude): {mae_lat}")
    print(f"Mean Absolute Error (Longitude): {mae_lon}")
    
def retrain(url, df, save_as="updated_model.pkl"):
    API_URL = f'{url}/retrain'
    
    data_payload = {
        "features": df[["depth", "mag", "gap", "dmin", "rms"]].to_dict(orient="records"),
        "latitude": df["latitude"].tolist(),
        "longitude": df["longitude"].tolist()
    }

    params = {"save_as": save_as}
    response = requests.post(API_URL, params=params, json=data_payload)
    print(response.json())
    
def list_models(url):
    print(requests.get(f'{url}/list_models').json())
    
def reload_model(url, model_name):
    API_URL = f'{url}/reload_model'
    params = {"model_name": model_name}

    response = requests.post(API_URL, params=params)
    print(response.json())
    
def download_model(url, model_name):
    API_URL = f'{url}/download_model'
    params = {"model_name": model_name}
    
    response = requests.get(API_URL, params=params)
    
    if response.status_code == 200:
        with open(model_name, "wb") as file:
            file.write(response.content)
        print(f"✅ {model_name} downloaded successfully!")
    else:
        print(f"❌ Failed to download {model_name}. Error: {response.json()}")
    
def upload_model(url, model_file):
    API_URL = f'{url}/upload_model'

    with open(model_file, "rb") as file:
        response = requests.post(API_URL, files={"file": file})
        
    print(response.json())

# Zone 1

In [ ]:
zone1_API = "https://fedml.onrender.com"
X_1, y_1 = load_val_data(zone_1_val_data)

In [ ]:
list_models(zone1_API)

{'saved_models': ['zone_0_model.pkl']}


In [ ]:
predictions = predict(zone1_API, X_1)

In [7]:
evaluate_predictions(predictions, y_1)

Mean Squared Error (Latitude): 55.6867846556177
Mean Squared Error (Longitude): 22.85913167833489

R² Score (Latitude): -5.6006510125206415
R² Score (Longitude): -21.468817233732548

Mean Absolute Error (Latitude): 6.683537563025212
Mean Absolute Error (Longitude): 4.655139546218472


In [ ]:
retrain(zone1_API, zone_1_training_data, 'zone_0_zone_1_model.pkl')

{'message': 'Model retrained and saved as zone0_zone1_model.pkl.'}


In [9]:
list_models(zone1_API)

{'saved_models': ['zone_0_model.pkl', 'zone0_zone1_model.pkl']}


In [ ]:
reload_model(zone1_API, 'zone_0_zone_1_model.pkl')

{'message': '✅ Model zone0_zone1_model.pkl successfully reloaded!'}


In [ ]:
predictions_1 = predict(zone1_API, X_1)

In [12]:
evaluate_predictions(predictions, y_1)

Mean Squared Error (Latitude): 2.4021747642564724
Mean Squared Error (Longitude): 0.9270269899674732

R² Score (Latitude): 0.7152660655845307
R² Score (Longitude): 0.0888017838377656

Mean Absolute Error (Latitude): 0.8764450252100842
Mean Absolute Error (Longitude): 0.5761097478991635


In [ ]:
download_model(zone1_API, 'zone_0_zone_1_model.pkl')

✅ zone0_zone1_model.pkl downloaded successfully!


# Zone 2

In [ ]:
zone2_API = "https://fedml-zone2.onrender.com"
X_2, y_2 = load_val_data(zone_2_val_data)

In [15]:
list_models(zone2_API)

{'saved_models': ['zone_0_model.pkl']}


In [16]:
predictions_2 = predict(zone2_API, X_2)

In [17]:
evaluate_predictions(predictions_2, y_2)

Mean Squared Error (Latitude): 8.581386455635915
Mean Squared Error (Longitude): 1.419967758709142

R² Score (Latitude): -12.975120181897791
R² Score (Longitude): -0.03546827145377596

Mean Absolute Error (Latitude): 2.598107553333334
Mean Absolute Error (Longitude): 0.8375822466666637


In [ ]:
retrain(zone2_API, zone_2_training_data, "zone_0_zone2_model.pkl")

{'message': 'Model retrained and saved as zone0_zone2_model.pkl.'}


In [29]:
list_models(zone2_API)

{'saved_models': ['zone_0_model.pkl', 'zone0_zone2_model.pkl']}


In [ ]:
reload_model(zone2_API, "zone_0_zone_2_model.pkl")

{'message': '✅ Model zone0_zone2_model.pkl successfully reloaded!'}


In [21]:
predictions_2 = predict(zone2_API, X_2)

In [22]:
evaluate_predictions(predictions_2, y_2)

Mean Squared Error (Latitude): 0.28959644008745367
Mean Squared Error (Longitude): 0.6505956206239898

R² Score (Latitude): 0.5283809818617452
R² Score (Longitude): 0.5255729444763328

Mean Absolute Error (Latitude): 0.36367409333333334
Mean Absolute Error (Longitude): 0.4909193933333293


In [ ]:
download_model(zone2_API, 'zone_0_zone_2_model.pkl')

✅ zone0_zone2_model.pkl downloaded successfully!


# Aggregation and Deployment of New Model

In [ ]:
from sklearn.ensemble import RandomForestRegressor
import joblib

In [49]:
zone_0_model = joblib.load("zone_0_model.pkl")
zone_1_model = joblib.load("zone0_zone1_model.pkl")
zone_2_model = joblib.load("zone0_zone2_model.pkl")

In [ ]:
import numpy as np
import joblib
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

# Extract decision boundary samples from a RandomForestRegressor.
def extract_decision_data(model):
    X_decision, y_decision = [], []
    
    for tree in model.estimators_:
        tree_ = tree.tree_
        features = tree_.feature
        thresholds = tree_.threshold
        values = tree_.value.squeeze()

        for node in range(tree_.node_count):
            if features[node] != -2:  # Not a leaf node
                X_decision.append(features[node])
                y_decision.append(thresholds[node])
    
    return np.array(X_decision).reshape(-1, 1), np.array(y_decision)

# Generates a global model by learning from decision boundaries of all local models.
def federated_decision_boundary_approximation(models):

    all_X, all_y = [], []

    for model in models:
        X_decision, y_decision = extract_decision_data(model)
        all_X.append(X_decision)
        all_y.append(y_decision)

    X_combined = np.vstack(all_X)
    y_combined = np.hstack(all_y)

    global_model = RandomForestRegressor(n_estimators=50)
    global_model.fit(X_combined, y_combined)

    return global_model

zone_0_model = joblib.load("zone_0_model.pkl")
zone_1_model = joblib.load("zone_0_zone_1_model.pkl")
zone_2_model = joblib.load("zone_0_zone_2_model.pkl")

agg_model = federated_decision_boundary_approximation([zone_0_model, zone_1_model, zone_2_model])
joblib.dump(agg_model, "zone_0+1+2_model.pkl")

print("Federated Decision Boundary Approximation completed. Model saved as zone_0+1+2_model.pkl")

Federated Decision Boundary Approximation completed. Model saved as federated_approx_model.pkl
